# 75.06/95.58 Organización de Datos
# Primer Cuatrimestre de 2020
# Trabajo Práctico 2: Enunciado


El segundo TP es una competencia de Machine Learning en donde cada grupo debe intentar determinar, para cada tweet brindado, si el mismo esta basado en un hecho real o no.

La competencia se desarrolla en la plataforma de Kaggle  https://www.kaggle.com/c/nlp-getting-started.  

El dataset consta de una serie de tweets, para los cuales se informa:

- id - identificador unico para cada  tweet
- text - el texto del tweet
- location - ubicación desde donde fue enviado (podría no estar)
- keyword - un keyword para el tweet  (podría faltar)
- target - en train.csv, indica si se trata de un desastre real  (1) o no (0)
 


Los submits con el resultado deben tener el formato:

Id: Un id numérico para identificar el tweet
target: 1 / 0 según se crea que el tweet se trata sobre un desastre real, o no.

Los grupos deberán probar distintos algoritmos de Machine Learning para intentar predecir si el tweet está basado en hechos reales o no. A medida que los grupos realicen pruebas deben realizar el correspondiente submit en Kaggle para evaluar el resultado de los mismos.

Al finalizar la competencia el grupo que mejor resultado tenga obtendrá 10 puntos para cada uno de sus integrantes que podrán ser usados en el examen por promoción o segundo recuperatorio.

## Requisitos para la entrega del TP2:

- El TP debe programarse en Python o R.
- Debe entregarse un pdf con el informe de algoritmos probados, algoritmo final utilizado, transformaciones realizadas a los datos, feature engineering, etc. 
- El informe debe incluir también un link a github con el informe presentado en pdf, y todo el código.
- El grupo debe presentar el TP en una computadora en la fecha indicada por la cátedra, el TP debe correr en un lapso de tiempo razonable (inferior a 1 hora) y generar un submission válido que iguale el mejor resultado obtenido por el grupo en Kaggle. (mas detalles a definir)

## El TP2 se va a evaluar en función del siguiente criterio:

- Cantidad de trabajo (esfuerzo) del grupo: ¿Probaron muchos algoritmos? ¿Hicieron un buen trabajo de pre-procesamiento de los datos y feature engineering?
- Resultado obtenido en Kaggle (obviamente cuanto mejor resultado mejor nota)
- Presentación final del informe, calidad de la redacción, uso de información obtenida en el TP1, conclusiones presentadas.
- Performance de la solución final.

## Forma de evaluar kaggle

Las presentaciones se evalúan utilizando F1 entre las respuestas previstas y esperadas.

F1 se calcula de la siguiente manera:
$ F1=(2∗precision∗recall)/(precision+recall) $

donde:

$precision=TP/(TP+FP)$

$recall=TP/(TP+FN)$


con:

- Verdadero positivo [TP] = your prediction is 1, and the ground truth is also 1 - you predicted a positive and that's true!
- Falso positivo [FP] = your prediction is 1, and the ground truth is 0 - you predicted a positive, and that's false.
- Falso Negativo [FN] = your prediction is 0, and the ground truth is 1 - you predicted a negative, and that's false.

importaciones

In [1]:
#IMPORTS
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go

#plt.style.use('default')

sns.set(style="whitegrid")

pd.options.display.float_format = '{:20,.2f}'.format 

import warnings
warnings.filterwarnings('ignore')

from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches
 
%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 9)
plt.style.use('ggplot')
 
#librerias para KNN
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
#red neuronal
from keras.models import Sequential
from keras.layers.core import Dense
#arboles de decicion
plt.rcParams['figure.figsize'] = (16, 9)
plt.style.use('ggplot')
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from IPython.display import Image as PImage
from subprocess import check_call
from PIL import Image, ImageDraw, ImageFont
#catboost
from catboost import CatBoostRegressor

Using TensorFlow backend.


inicio de csv

In [2]:
tweets_original_entrenamiento = pd.read_csv('train.csv',index_col=['id'])
tweets_original_entrenamiento.head()

,keyword,location,text,target
id,,,,
1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [3]:
tweets_original_prueva = pd.read_csv('test.csv',index_col=['id'])
tweets_original_prueva.head()

,keyword,location,text
id,,,
0,NaN,NaN,Just happened a terrible car crash
2,NaN,NaN,"Heard about #earthquake is different cities, s..."
3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


calculo resultado

In [4]:
resultados_de_test = pd.read_csv('sample_submission.csv',index_col=['id'])

def calclarPresicionYRecall(df):
    df['TP'] = (df['target'] and resultados_de_test['target'])
    df['FP'] = ((not df['target']) and resultados_de_test['target'])
    df['FN'] = (df['target'] and (not resultados_de_test['target']))
    tp = df['TP'].sum
    fp = df['FP'].sum
    fn = df['FM'].sum
    presicion = tp / (tp + fp)
    recall = tp / (tp + fn)
    return (presicion,recall)

def F1(df):
    presicion,recall = calclarPresicionYRecall(df)
    resultado = 2 * presicion * recall
    resultado /= (presicion + recall)
    return resultado

inicio de problema

## CATBOOST

In [35]:
train_label = tweets_original_entrenamiento['target']
train_set = tweets_original_entrenamiento.drop(columns=['target'])

train_set['text'].fillna("", inplace=True)
train_set['keyword'].fillna("", inplace=True)
train_set['location'].fillna("", inplace=True)

train_set['longitud_de_texto'] = train_set['text'].str.len()
train_set['longitud_de_keyword'] = train_set['keyword'].str.len()
train_set['cantidad_de_palabras_texto'] = train_set['text'].str.split().map(len)
train_set['cantidad_de_palabras_keyword'] = train_set['keyword'].str.split().map(len)

#train_set['text'] = train_set['text'].str.replace(r' ', '')
#train_set['keyword'] = train_set['keyword'].str.replace(r' ', '')
#train_set['location'] = train_set['location'].str.replace(r' ', '')

train_set = train_set.drop(columns=['keyword','text','location'])
train_set.head()

,longitud_de_texto,longitud_de_keyword,cantidad_de_palabras_texto,cantidad_de_palabras_keyword
id,,,,
1,69,0,13,0
4,38,0,7,0
5,133,0,22,0
6,65,0,8,0
7,88,0,16,0


In [34]:
test_set = tweets_original_prueva

test_set['text'].fillna("", inplace=True)
test_set['keyword'].fillna("", inplace=True)
test_set['location'].fillna("", inplace=True)

test_set['longitud_de_texto'] = test_set['text'].str.len()
test_set['longitud_de_keyword'] = test_set['keyword'].str.len()
test_set['cantidad_de_palabras_texto'] = test_set['text'].str.split().map(len)
test_set['cantidad_de_palabras_keyword'] = test_set['keyword'].str.split().map(len)

#test_set['text'] = test_set['text'].str.replace(r' ', '')
#test_set['keyword'] = test_set['keyword'].str.replace(r' ', '')
#test_set['location'] = test_set['location'].str.replace(r' ', '')

test_set = test_set.drop(columns=['keyword','text','location'])
test_set.head()

,longitud_de_texto,longitud_de_keyword,cantidad_de_palabras_texto,cantidad_de_palabras_keyword
id,,,,
0,29,0,1,0
2,56,0,1,0
3,78,0,1,0
9,37,0,1,0
11,38,0,1,0


In [49]:
# Initialize CatBoostRegressor
model = CatBoostRegressor(iterations=5,
                          learning_rate=0.3,
                          depth=3)
# Fit model
model.fit(train_set, train_label)
# Get predictions
predicion = model.predict(test_set)

0:	learn: 0.4868874	total: 1.79ms	remaining: 7.17ms
1:	learn: 0.4823555	total: 3.27ms	remaining: 4.9ms
2:	learn: 0.4795201	total: 4.78ms	remaining: 3.18ms
3:	learn: 0.4767718	total: 6.3ms	remaining: 1.57ms
4:	learn: 0.4750395	total: 7.82ms	remaining: 0us


In [59]:
#resultado
solucionCatBoost =pd.DataFrame(data=predicion)
solucionCatBoost.columns = ['target']
solucionCatBoost['target'] = solucionCatBoost['target'].round()
#solucionCatBoost.head()
resultado = F1(solucionCatBoost)
print(resultado)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

## ARBOL DE DECISION

In [ ]:
#para arbol verificar si esta balanceado o no
train_set.groupby(["target"]).size()